In [2]:
import requests
url = "https://www.kaggle.com/datasets/carlmcbrideellis/llm-7-prompt-training-dataset/"
response = requests.get(url)

In [3]:
from bs4 import BeautifulSoup
soup= BeautifulSoup(response.text, 'html.parser')

In [ ]:
file_url=soup.find

In [26]:
# from selenium import webdriver
# import time
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.chrome.options import Options
# s=Service("C:/Users/gksxo/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe")
# options = Options()
# driver = webdriver.Chrome(service=s, options=options)
# WebDriver 경로 설정
# driver = webdriver.Chrome("C:/Users/gksxo/Downloads/chromedriver_win32 (1)/chromedriver.exe")

# 웹 페이지 접근
# url = 'https://www.kaggle.com/account/login?phase=emailSignIn&returnUrl=%2F'
# driver.get(url)
 # *[@id="site-content"]/div[2]/div/div/div[2]/div[1]/div/a/button/i/font/font
 # //*[@id="site-content"]/div[2]/div/div/div[2]/div[1]/div/a/button
#로그인이 필요한 경우 아래 코드 활성화
#driver.find_element_by_xpath("//button[@data-testid='headerSignInButton']").click()
#time.sleep(2)  # 로딩 대기
#driver.find_element_by_xpath("//input[@name='username']").send_keys('id')
#driver.find_element_by_xpath("//input[@name='password']").send_keys('pw')
#driver.find_element_by_xpath("//button[@data-testid='signInSubmit']").click()
#time.sleep(2)  # 로딩 대기

# 파일 다운로드 버튼 클릭
#download_btn = driver.find_element_by_xpath("//button[@class='sc-hjsqBZ bQkJuY sc-iiBYEw hcSHvu']")
#download_btn.click()

# WebDriver 종료
#driver.quit()


In [2]:
import pandas as pd
import numpy as np

In [3]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

a=api.dataset_download_files('carlmcbrideellis/llm-7-prompt-training-dataset')
# os.system("unzip -v C:/Users/gksxo/Desktop/2023학년도 2학기/오픈소스-조겨리 교수님/기말프로젝트/llm-7-prompt-training-dataset.zip")

import zipfile

zip_path = 'C:/Users/gksxo/Desktop/2023학년도 2학기/오픈소스-조겨리 교수님/기말프로젝트/llm-7-prompt-training-dataset.zip'  # 압축 파일 경로
extract_path = 'C:/Users/gksxo/Desktop/2023학년도 2학기/오픈소스-조겨리 교수님/기말프로젝트'  # 압축 해제 경로

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [4]:
prompt=pd.read_csv("C:/Users/gksxo/Desktop/2023학년도 2학기/오픈소스-조겨리 교수님/기말프로젝트/train_essays_7_prompts_v2.csv")
prompt.head()

,text,label
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0


In [5]:
prompt.count()

text     15350
label    15350
dtype: int64

In [7]:
prompt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15350 entries, 0 to 15349
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    15350 non-null  object
 1   label   15350 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 240.0+ KB


In [8]:
prompt.describe()

,label
count,15350.000000
mean,0.106710
std,0.308754
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,1.000000


In [9]:
import sklearn

In [11]:
prompt.groupby("label")

In [12]:
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

ModuleNotFoundError: No module named 'torch'